# OpenLineage + Spark (Marquez Postgres Report)

This notebook reads Marquez tables directly from Docker Postgres using Spark JDBC and prepares a report for:
- most recent OpenLineage activities
- current known assets
- unique source assets, destination assets, and source-to-destination pairs

Expected Docker Compose defaults:
- host: `postgres`
- db: `marquez`
- user/password: `marquez` / `marquez`


In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T


In [2]:
# JDBC connection settings (override if needed)
POSTGRES_HOST = "postgres"
POSTGRES_PORT = "5432"
POSTGRES_DB = "marquez"
POSTGRES_USER = "marquez"
POSTGRES_PASSWORD = "marquez"

JDBC_URL = f"jdbc:postgresql://{POSTGRES_HOST}:{POSTGRES_PORT}/{POSTGRES_DB}"
JDBC_PROPS = {
    "user": POSTGRES_USER,
    "password": POSTGRES_PASSWORD,
    "driver": "org.postgresql.Driver",
}

spark = (
    SparkSession.builder
    .appName("openlineage-marquez-postgres-report")
    .config("spark.jars.packages", "org.postgresql:postgresql:42.7.3")
    .getOrCreate()
)

spark.version


'3.5.3'

In [3]:
# Helpers

def read_sql(query: str):
    return (
        spark.read
        .format("jdbc")
        .option("url", JDBC_URL)
        .option("driver", JDBC_PROPS["driver"])
        .option("user", JDBC_PROPS["user"])
        .option("password", JDBC_PROPS["password"])
        .option("dbtable", f"({query}) as q")
        .load()
    )


def read_table(table_name: str, schema: str = "public"):
    return (
        spark.read
        .format("jdbc")
        .option("url", JDBC_URL)
        .option("driver", JDBC_PROPS["driver"])
        .option("user", JDBC_PROPS["user"])
        .option("password", JDBC_PROPS["password"])
        .option("dbtable", f"{schema}.{table_name}")
        .load()
    )


def pick_first_existing(candidates, columns):
    for c in candidates:
        if c in columns:
            return c
    return None


In [4]:
# Discover available Marquez tables in public schema
available_tables_df = read_sql("""
    SELECT table_name
    FROM information_schema.tables
    WHERE table_schema = 'public'
    ORDER BY table_name
""")

available_tables = [r["table_name"] for r in available_tables_df.collect()]
print(f"Found {len(available_tables)} tables")
for t in available_tables:
    print(" -", t)


Found 37 tables
 - column_lineage
 - dataset_facets
 - dataset_facets_view
 - dataset_fields
 - dataset_fields_tag_mapping
 - dataset_schema_versions
 - dataset_schema_versions_field_mapping
 - dataset_symlinks
 - dataset_versions
 - dataset_versions_field_mapping
 - datasets
 - datasets_tag_mapping
 - datasets_view
 - facet_migration_lock
 - flyway_schema_history
 - job_facets
 - job_facets_view
 - job_versions
 - job_versions_io_mapping
 - jobs
 - jobs_fqn
 - jobs_tag_mapping
 - jobs_view
 - lineage_events
 - namespace_ownerships
 - namespaces
 - owners
 - run_args
 - run_facets
 - run_facets_view
 - run_states
 - runs
 - runs_input_mapping
 - runs_view
 - sources
 - stream_versions
 - tags


In [6]:
# Read key tables if present
lineage_events_table = pick_first_existing(["lineage_events", "events"], available_tables)
datasets_table = pick_first_existing(["datasets"], available_tables)

if lineage_events_table is None:
    raise RuntimeError("Could not find lineage events table (expected: lineage_events or events)")
if datasets_table is None:
    raise RuntimeError("Could not find datasets table")

lineage_events_df = read_table(lineage_events_table)
datasets_df = read_table(datasets_table)

print("lineage events table:", lineage_events_table)
print("datasets table:", datasets_table)
print("lineage events rows:", lineage_events_df.count())
print("datasets rows:", datasets_df.count())


lineage events table: lineage_events
datasets table: datasets
lineage events rows: 67
datasets rows: 14


In [7]:
# Parse lineage events into a normalized activity frame
lineage_cols = set(lineage_events_df.columns)

event_json_col = pick_first_existing(["event", "event_json", "payload", "lineage_event"], lineage_cols)
event_time_col = pick_first_existing(["event_time", "created_at", "updated_at", "transitioned_at"], lineage_cols)
event_type_col = pick_first_existing(["event_type", "type"], lineage_cols)

if event_json_col is None:
    raise RuntimeError("Could not find JSON event payload column in lineage events table")

activity_df = lineage_events_df

if event_time_col is not None:
    activity_df = activity_df.withColumn("activity_time", F.col(event_time_col).cast("timestamp"))
else:
    activity_df = activity_df.withColumn("activity_time", F.current_timestamp())

if event_type_col is not None:
    activity_df = activity_df.withColumn("activity_type", F.col(event_type_col).cast("string"))
else:
    activity_df = activity_df.withColumn("activity_type", F.get_json_object(F.col(event_json_col).cast("string"), "$.eventType"))

activity_df = (
    activity_df
    .withColumn("event_json", F.col(event_json_col).cast("string"))
    .withColumn("job_namespace", F.get_json_object(F.col("event_json"), "$.job.namespace"))
    .withColumn("job_name", F.get_json_object(F.col("event_json"), "$.job.name"))
    .withColumn("run_id", F.get_json_object(F.col("event_json"), "$.run.runId"))
)

recent_activities_df = (
    activity_df
    .select("activity_time", "activity_type", "job_namespace", "job_name", "run_id")
    .orderBy(F.col("activity_time").desc_nulls_last())
)

recent_activities_df.show(50, truncate=False)


+-----------------------+-------------+-------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------+
|activity_time          |activity_type|job_namespace|job_name                                                                                                                                                                   |run_id                              |
+-----------------------+-------------+-------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------+
|2026-02-24 09:35:11.081|COMPLETE     |local-spark  |openlineage_marquez_postgres_report.scan_jdbc_relation_select_table_name_from_information_schema_tables_where_table_schema_public_order_by_table_name_as_q_num

In [9]:
# Current assets from datasets table (with flexible column mapping)
dataset_cols = set(datasets_df.columns)

asset_name_col = pick_first_existing(["name", "dataset_name"], dataset_cols)
asset_namespace_col = pick_first_existing(["namespace_name", "namespace", "current_namespace_name"], dataset_cols)
asset_type_col = pick_first_existing(["type", "physical_name"], dataset_cols)
asset_updated_col = pick_first_existing(["updated_at", "created_at"], dataset_cols)

if asset_name_col is None:
    raise RuntimeError("Could not find dataset name column in datasets table")

current_assets_df = (
    datasets_df
    .select(
        (F.col(asset_namespace_col) if asset_namespace_col else F.lit("unknown")).alias("namespace"),
        F.col(asset_name_col).alias("asset_name"),
        (F.col(asset_type_col) if asset_type_col else F.lit(None)).alias("asset_type"),
        (F.col(asset_updated_col).cast("timestamp") if asset_updated_col else F.lit(None).cast("timestamp")).alias("last_seen_at"),
    )
    .dropDuplicates(["namespace", "asset_name"])
    .orderBy(F.col("last_seen_at").desc_nulls_last(), F.col("namespace"), F.col("asset_name"))
)

current_assets_df.show(200, truncate=False)
print("Current unique assets:", current_assets_df.count())


+------------------------+-------------------------------------------------------------+----------+--------------------------+
|namespace               |asset_name                                                   |asset_type|last_seen_at              |
+------------------------+-------------------------------------------------------------+----------+--------------------------+
|postgres://postgres:5432|marquez.public.datasets                                      |DB_TABLE  |2026-02-24 09:35:38.036   |
|postgres://postgres:5432|marquez.public.lineage_events                                |DB_TABLE  |2026-02-24 09:35:29.171   |
|postgres://postgres:5432|marquez.information_schema.tables                            |DB_TABLE  |2026-02-24 09:35:11.081   |
|file                    |/home/jovyan/work/data/complex_demo/gold/customer_360        |DB_TABLE  |2026-02-24 09:19:38.774001|
|file                    |/home/jovyan/work/data/complex_demo/gold/product_performance |DB_TABLE  |2026-02-24 0

In [10]:
# Unique sources and destinations from lineage event inputs/outputs
io_schema = T.ArrayType(
    T.StructType([
        T.StructField("namespace", T.StringType(), True),
        T.StructField("name", T.StringType(), True),
    ])
)

io_df = (
    activity_df
    .select(
        "activity_time",
        F.from_json(F.get_json_object(F.col("event_json"), "$.inputs"), io_schema).alias("inputs"),
        F.from_json(F.get_json_object(F.col("event_json"), "$.outputs"), io_schema).alias("outputs"),
    )
)

source_assets_df = (
    io_df
    .select(F.explode_outer("inputs").alias("ds"))
    .select(F.col("ds.namespace").alias("namespace"), F.col("ds.name").alias("asset"))
    .filter(F.col("asset").isNotNull())
    .dropDuplicates(["namespace", "asset"])
    .orderBy("namespace", "asset")
)

destination_assets_df = (
    io_df
    .select(F.explode_outer("outputs").alias("ds"))
    .select(F.col("ds.namespace").alias("namespace"), F.col("ds.name").alias("asset"))
    .filter(F.col("asset").isNotNull())
    .dropDuplicates(["namespace", "asset"])
    .orderBy("namespace", "asset")
)

source_to_destination_df = (
    io_df
    .withColumn("src", F.explode_outer("inputs"))
    .withColumn("dst", F.explode_outer("outputs"))
    .select(
        F.col("src.namespace").alias("source_namespace"),
        F.col("src.name").alias("source_asset"),
        F.col("dst.namespace").alias("destination_namespace"),
        F.col("dst.name").alias("destination_asset"),
    )
    .filter(F.col("source_asset").isNotNull() & F.col("destination_asset").isNotNull())
    .dropDuplicates([
        "source_namespace", "source_asset", "destination_namespace", "destination_asset"
    ])
    .orderBy("source_namespace", "source_asset", "destination_namespace", "destination_asset")
)

print("Unique sources:", source_assets_df.count())
print("Unique destinations:", destination_assets_df.count())
print("Unique source->destination pairs:", source_to_destination_df.count())

print("Sources")
source_assets_df.show(200, truncate=False)

print("Destinations")
destination_assets_df.show(200, truncate=False)

print("Source -> Destination")
source_to_destination_df.show(200, truncate=False)


Unique sources: 8
Unique destinations: 6
Unique source->destination pairs: 18
Sources
+------------------------+--------------------------------------------------------+
|namespace               |asset                                                   |
+------------------------+--------------------------------------------------------+
|file                    |/home/jovyan/work/data/complex_demo/source/customers.csv|
|file                    |/home/jovyan/work/data/complex_demo/source/fx_rates.csv |
|file                    |/home/jovyan/work/data/complex_demo/source/orders.csv   |
|file                    |/home/jovyan/work/data/complex_demo/source/products.csv |
|file                    |/home/jovyan/work/data/people.csv                       |
|postgres://postgres:5432|marquez.information_schema.tables                       |
|postgres://postgres:5432|marquez.public.datasets                                 |
|postgres://postgres:5432|marquez.public.lineage_events                   

In [11]:
# Optional: compact KPI summary + persist as timestamped CSV
from datetime import datetime
import csv
import os

summary_df = spark.createDataFrame([
    ("recent_activity_rows", recent_activities_df.count()),
    ("current_assets", current_assets_df.count()),
    ("unique_sources", source_assets_df.count()),
    ("unique_destinations", destination_assets_df.count()),
    ("unique_source_destination_pairs", source_to_destination_df.count()),
], ["metric", "value"])

summary_df.show(truncate=False)

report_dir = "/home/jovyan/work/reports"
os.makedirs(report_dir, exist_ok=True)

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
summary_csv_path = os.path.join(report_dir, f"openlineage_summary_{timestamp}.csv")

with open(summary_csv_path, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["metric", "value"])
    for row in summary_df.collect():
        writer.writerow([row["metric"], row["value"]])

print("Summary CSV saved to:", summary_csv_path)
summary_csv_path


+-------------------------------+-----+
|metric                         |value|
+-------------------------------+-----+
|recent_activity_rows           |90   |
|current_assets                 |14   |
|unique_sources                 |8    |
|unique_destinations            |6    |
|unique_source_destination_pairs|18   |
+-------------------------------+-----+

Summary CSV saved to: /home/jovyan/work/reports/openlineage_summary_20260224_093548.csv


'/home/jovyan/work/reports/openlineage_summary_20260224_093548.csv'

## Notes

- If JDBC driver resolution fails, restart the kernel and run from the first cell.
- In Docker Compose, run this notebook from the `notebook` container so host `postgres` resolves correctly.
- You can filter `recent_activities_df` to a time window for focused investigations.
